In [37]:
!pip install boto3

     |████████████████████████████████| 131 kB 6.1 MB/s eta 0:00:01
     |████████████████████████████████| 7.5 MB 12.5 MB/s eta 0:00:01
  Using cached jmespath-0.10.0-py2.py3-none-any.whl (24 kB)
  Using cached s3transfer-0.4.2-py2.py3-none-any.whl (79 kB)


In [78]:
import os
import h5py
import time
import argparse
import copy

import numpy as np
from tqdm import tqdm
import logging
import boto3
from botocore.exceptions import ClientError
import matplotlib.pyplot as plt
from PIL import Image
import csv
from tqdm import tqdm

In [75]:
creds = {}
with open('../data/rootkey.csv', newline='') as rootkey:
    reader = csv.reader(rootkey, delimiter=' ')
    for row in reader:
        key, value = row[0].split("=")
        creds[key] = value
print(creds.keys())

BUCKET_NAME = 'kiln-labeling'

dict_keys(['AWSAccessKeyId', 'AWSSecretKey'])


In [85]:
DATA_PATH = '../data/mturk_test_hdfs' if local_mode else '/atlas/u/mhelabd/data/kiln-scaling/labelled_bangladesh_2019_2020/negative_sample/'
directory = os.fsencode(DATA_PATH)
all_files = os.listdir(directory)

print("directory: {}".format(directory))
print("all files ({}): {}".format(len(all_files), all_files))

directory: b'../data/mturk_test_hdfs'
all files (5): [b'examples_1.hdf5', b'examples_2', b'.ipynb_checkpoints', b'examples_2.hdf5', b'examples_1']


In [2]:
local_mode = True

In [66]:
fig_index = 0

def visualize_tile(image, indices=[3, 2, 1]):
    global fig_index
    plt.figure(fig_index)
    fig_index += 1
    row_idx = np.array(indices)
    X = np.transpose(image[row_idx], (1, 2, 0))
    X *= 1 / np.max(X)
    plt.imshow(X)
    
def save_tile_as_image(data, img_filename, indices=[3, 2, 1]):
    X = np.transpose(data, (1, 2, 0))
    X *= 1 / np.max(X) # * 255
    img = Image.fromarray((X * 255).astype(np.uint8))
    img.save(img_filename, "JPEG")

In [76]:
# Retrieve the list of existing buckets
s3 = boto3.client(
    's3',
    aws_access_key_id=creds['AWSAccessKeyId'],
    aws_secret_access_key=creds['AWSSecretKey'],
)
response = s3.list_buckets()

# Output the bucket names
print('Existing buckets:')
for bucket in response['Buckets']:
    print(f'  {bucket["Name"]}')
    
def upload_file(file_path, bucket_name, obj_name):
    global s3
    try:
        response = s3.upload_file(
            file_path, 
            bucket_name, 
            obj_name,
            ExtraArgs={'ACL': 'public-read', 'ContentType': 'image/jpeg'}
        )
    except ClientError as e:
        logging.error(e)
        return False
    return True

Existing buckets:
  kiln-labeling


In [86]:
for file_index, file in enumerate(all_files):
    FILENAME = os.fsdecode(file)
    if FILENAME[-5:] == '.hdf5':
        print("Starting file: " + FILENAME)
        with h5py.File(f"{DATA_PATH}/{FILENAME}", 'r') as f:
            folder_name = FILENAME[:-5]
            if not os.path.exists(DATA_PATH + "/" + folder_name):
                os.makedirs(DATA_PATH + "/" + folder_name)

            images = np.array(f['images'])
            for index in tqdm(range(images.shape[0])):
                # visualize_tile(images[index])
                rgb_image = images[index][[3, 2, 1], :, :]
                # check images are not nan
                if not np.isnan(rgb_image).any():
                    local_filename = DATA_PATH + "/" + folder_name + "/" + str(index) + ".jpeg"
                    # save local file if it doesn't exist already
                    if not os.path.exists(local_filename):
                        save_tile_as_image(rgb_image, local_filename)
                    # upload to s3
                    s3_filename = folder_name + "/" + str(index) + ".jpeg"
                    upload_file(local_filename, BUCKET_NAME, s3_filename)

Starting file: examples_1.hdf5


100%|██████████| 999/999 [01:47<00:00,  9.26it/s]


Starting file: examples_2.hdf5


100%|██████████| 999/999 [02:41<00:00,  6.20it/s]
